In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# 머신러닝을 지원하는 sklearn 라이브러리 가져오기
from sklearn.neighbors import KNeighborsClassifier
# 평가 지표들을 활용할 수 있는 metrics 모듈을 불러오기
from sklearn import metrics
import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split#학습 평가 데이터 분리
import seaborn as sns #matplot과 유사하게 시각화 라이브러리이지만 좀 더 깔끔함
from sklearn.datasets import load_iris #붓꽃 iris
from sklearn.tree import DecisionTreeClassifier #트리
# !pip install seaborn
# 손쉽게 레이블 인코딩을 하기 위해 레이블인코더를 임포트
from sklearn.preprocessing import LabelEncoder
#!pip install graphviz #시각화 외부라이브러리
from sklearn.tree import export_graphviz
from sklearn.preprocessing import StandardScaler #StandardScaler
from sklearn.preprocessing import MinMaxScaler  #MinMaxScaler
from sklearn.datasets import load_breast_cancer
#!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 목차보기
from sklearn.model_selection import cross_val_score#교차검증 적용을 위해 임포트
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier #앙상블 모델
from sklearn.ensemble import AdaBoostClassifier # 에이다
#!pip install xgboost #XG부스트
#!pip install lightgbm #LightGBM
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# 빨간색 경고창 무시하기
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split #트레인테스트 스플릿
from sklearn.model_selection import GridSearchCV # 그리드서치
from sklearn.model_selection import KFold # cv = kfold
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression#선형회귀 모델 임포트
from sklearn.datasets import load_boston #학습용 보스턴 집 값 데이터를 임포트
from sklearn.metrics import mean_squared_error # mse
from sklearn.linear_model import Lasso # Lasso 임포트
from sklearn.linear_model import Ridge # Ridge 임포트
from sklearn.linear_model import LogisticRegression #로지스틱 임포트
# 한글을 쓰기 위한 코드
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname ="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family = font_name)
from surprise import Reader, Dataset
from surprise import SVD, accuracy

In [31]:
#critic(user)  title(item) rating
rating = pd.read_csv("data/movie_rating.csv")
rating

,critic,title,rating
0,Jack,Lady,3.0
1,Jack,Snakes,4.0
2,Jack,You Me,3.5
3,Jack,Superman,5.0
4,Jack,The Night,3.0
5,Mick,Lady,3.0
6,Mick,Snakes,4.0
7,Mick,Just My,2.0
8,Mick,Superman,3.0
9,Mick,You Me,2.0


In [16]:

rating['title'].value_counts()

Superman     6
You Me       6
Snakes       6
The Night    5
Lady         4
Just My      4
Name: title, dtype: int64

In [17]:
rating["critic"].value_counts()

Mick       6
Gene       6
Lisa       6
Jack       5
Claudia    5
Toby       3
Name: critic, dtype: int64

In [18]:
#관람vs미관람
tab = pd.crosstab(rating['critic'], rating['title'])
tab

title,Just My,Lady,Snakes,Superman,The Night,You Me
critic,,,,,,
Claudia,1,0,1,1,1,1
Gene,1,1,1,1,1,1
Jack,0,1,1,1,1,1
Lisa,1,1,1,1,1,1
Mick,1,1,1,1,1,1
Toby,0,0,1,1,0,1


In [19]:
# movie rating
# 두 개의 집단변수를 가지고 나머지 rating을 그룹화
# critic = 유저이름 , title = 영화이름
rating_g = rating.groupby(['critic', 'title'])
rating_g.sum()

rating
critic  title            
Claudia Just My       3.0
        Snakes        3.5
        Superman      4.0
        The Night     4.5
        You Me        2.5
Gene    Just My       1.5
        Lady          3.0
        Snakes        3.5
        Superman      5.0
        The Night     3.0
        You Me        3.5
Jack    Lady          3.0
        Snakes        4.0
        Superman      5.0
        The Night     3.0
        You Me        3.5
Lisa    Just My       3.0
        Lady          2.5
        Snakes        3.5
        Superman      3.5
        The Night     3.0
        You Me        2.5
Mick    Just My       2.0
        Lady          3.0
        Snakes        4.0
        Superman      3.0
        The Night     3.0
        You Me        2.0
Toby    Snakes        4.5
        Superman      4.0
        You Me        1.0

In [21]:
# 행렬구조로 변환
tab = rating_g.sum().unstack() 
tab

rating                                      
title   Just My Lady Snakes Superman The Night You Me
critic                                               
Claudia     3.0  NaN    3.5      4.0       4.5    2.5
Gene        1.5  3.0    3.5      5.0       3.0    3.5
Jack        NaN  3.0    4.0      5.0       3.0    3.5
Lisa        3.0  2.5    3.5      3.5       3.0    2.5
Mick        2.0  3.0    4.0      3.0       3.0    2.0
Toby        NaN  NaN    4.5      4.0       NaN    1.0

-Toby가 미관람한 영화 : 'Just My', 'Lady', 'The Night'

- rating 데이터셋 생성

In [22]:
reader = Reader(rating_scale= (1, 5)) # 평점 범위
data = Dataset.load_from_df(df=rating, reader=reader)
# rating이라는 데이터프레임은 reader(1~5)의 평점 범위를 가진다.
data

- train/test set

In [23]:
train = data.build_full_trainset() # 훈련셋
test = train.build_testset() # 검정셋

- model 생성

In [24]:
help(SVD)

model = SVD(n_factors=100, n_epochs=20, random_state=123)
model.fit(train) # model 생성

Help on class SVD in module surprise.prediction_algorithms.matrix_factorization:

class SVD(surprise.prediction_algorithms.algo_base.AlgoBase)
 |  SVD(n_factors=100, n_epochs=20, biased=True, init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)
 |  
 |  The famous *SVD* algorithm, as popularized by `Simon Funk
 |  <http://sifter.org/~simon/journal/20061211.html>`_ during the Netflix
 |  Prize. When baselines are not used, this is equivalent to Probabilistic
 |  Matrix Factorization :cite:`salakhutdinov2008a` (see :ref:`note
 |  <unbiased_note>` below).
 |  
 |  The prediction :math:`\hat{r}_{ui}` is set as:
 |  
 |  .. math::
 |      \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u
 |  
 |  If user :math:`u` is unknown, then the bias :math:`b_u` and the factors
 |  :math:`p_u` are assumed to be zero. The same applies for item :math:`i`
 |  with :math:`b_i

- Toby 사용자 영화추천

In [30]:
user_id = 'Toby' # 추천대상자
item_ids = ['The Night', 'Just My', 'Lady'] # 추천 대상 영화
actual_rating = 0 # 실제 평점

#est = 평점

for item_id in item_ids :
    print(model.predict(user_id, item_id, actual_rating))

user: Toby       item: The Night  r_ui = 0.00   est = 3.30   {'was_impossible': False}
user: Toby       item: Just My    r_ui = 0.00   est = 2.88   {'was_impossible': False}
user: Toby       item: Lady       r_ui = 0.00   est = 3.27   {'was_impossible': False}
